In [9]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor

df = pd.read_csv('result/data/melting_point_features.csv')

y = df['Tm']
X = df.drop(columns=['Tm'])

X = X.select_dtypes(include=[np.number])
X.replace([np.inf, -np.inf], np.nan, inplace=True)

imputer = SimpleImputer(strategy='median')
X_clean = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

X_train, X_test, y_train, y_test = train_test_split(X_clean, y, test_size=0.2, random_state=42)

base_model = LGBMRegressor(random_state=42, n_jobs=-1)

In [10]:
from sklearn.feature_selection import RFE
import time

print("\n---START RFE ---")
start = time.time()

model = LGBMRegressor(random_state=42, n_jobs=-1, verbose=-1)

rfe = RFE(estimator=model, n_features_to_select=30, step=0.1)

rfe.fit(X_train, y_train)

selected_rfe = X_train.columns[rfe.support_]
print(f"⏱️ Time Run: {time.time() - start:.2f} s")
print(f"✅ RFE Choosen {len(selected_rfe)} features:")
print(list(selected_rfe))


---START RFE ---
⏱️ Time Run: 5.65 s
✅ RFE Choosen 30 features:
['MinEStateIndex', 'qed', 'SPS', 'MolWt', 'NumValenceElectrons', 'MaxPartialCharge', 'MinPartialCharge', 'MinAbsPartialCharge', 'BCUT2D_MRHI', 'BalabanJ', 'Chi0n', 'Chi0v', 'HallKierAlpha', 'Kappa2', 'Kappa3', 'PEOE_VSA14', 'SMR_VSA10', 'SlogP_VSA2', 'TPSA', 'EState_VSA2', 'Phi', 'MolMR', 'SlogP_VSA0', 'SMR_VSA0', 'Gasteiger_q_std', 'HeteroAtomFrac', 'Flexibility_Score', 'Complexity_per_MW', 'FracSingle', 'FracDouble']


In [ ]:
import warnings
warnings.filterwarnings('ignore')
from sklearn_genetic import GAFeatureSelectionCV

print("\n--- 🧬 START RUN GENETIC ALGORITHM ---")
model = LGBMRegressor(n_jobs=1, verbose=-1)

ga = GAFeatureSelectionCV(
    estimator=model_dynamic,
    cv=3,                     
    scoring="neg_root_mean_squared_error",
    population_size=30,         
    generations=10,         
    mutation_probability=0.3,
    n_jobs=2,
    verbose=True              
)

ga.fit(X_train, y_train)

selected_ga = X_train.columns[ga.support_]

print(f"⏱️ Time Run: {time.time() - start:.2f} s")
print(f"\n✅ GA Choosen {len(selected_ga)} features:")
print(list(selected_ga))


--- 🧬 START RUN GENETIC ALGORITHM ---
gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	50    	-282.316	4.19157    	-272.724   	-293.101   
1  	27    	-278.404	3.61529    	-272.097   	-285.971   
2  	28    	-275.629	2.91447    	-271.178   	-284.259   
3  	22    	-273.852	1.75353    	-270.676   	-278.587   
4  	28    	-273.342	1.63655    	-270.676   	-277.56    
5  	30    	-272.897	1.99952    	-268.98    	-278.328   
6  	31    	-272.229	2.38925    	-268.98    	-281.741   
7  	28    	-271.119	1.82429    	-268.253   	-278.895   
8  	28    	-270.971	1.96509    	-268.253   	-276.154   
9  	32    	-270.292	2.32939    	-267.944   	-280.111   
10 	27    	-269.719	1.58393    	-267.944   	-274.849   
11 	28    	-269.375	1.25118    	-267.806   	-273.15    
12 	26    	-268.877	1.28269    	-266.555   	-273.083   
13 	24    	-268.701	1.73371    	-266.555   	-274.323   
14 	34    	-268.724	2.47547    	-266.555   	-279.985   
15 	25    	-268.078	2.07818    	-266.555   	-275.205   
16 	32   

In [12]:
common_features = set(selected_rfe) & set(selected_ga)
print(f"\n💎 Các features quan trọng được cả 2 thuật toán cùng chọn ({len(common_features)}):")
print(common_features)


💎 Các features quan trọng được cả 2 thuật toán cùng chọn (19):
{'Gasteiger_q_std', 'Flexibility_Score', 'MolMR', 'EState_VSA2', 'TPSA', 'SlogP_VSA2', 'SMR_VSA10', 'MolWt', 'FracDouble', 'NumValenceElectrons', 'BCUT2D_MRHI', 'qed', 'MinEStateIndex', 'SPS', 'FracSingle', 'Kappa3', 'HallKierAlpha', 'Chi0v', 'BalabanJ'}


In [13]:
best_features = selected_ga


final_model = LGBMRegressor(random_state=42, n_jobs=-1, verbose=-1)
final_model.fit(X_clean[best_features], y)

print(f"{len(best_features)} features.")

import joblib


joblib.dump(final_model, 'final_melting_point_model.pkl')
joblib.dump(list(best_features), 'final_features_list.pkl')

439 features.


['final_features_list.pkl']

In [14]:
import pandas as pd
features_to_save = list(selected_ga)

train_df = X_train[features_to_save].copy()
train_df['Tm'] = y_train

test_df = X_test[features_to_save].copy()
test_df['Tm'] = y_test

train_df.to_csv('train_data_reduced.csv', index=False)
test_df.to_csv('test_data_reduced.csv', index=False)

print(f"✅ Đã lưu dữ liệu rút gọn:")
print(f"   - Train: {train_df.shape} -> 'train_data_reduced.csv'")
print(f"   - Test:  {test_df.shape}  -> 'test_data_reduced.csv'")

✅ Đã lưu dữ liệu rút gọn:
   - Train: (8416, 440) -> 'train_data_reduced.csv'
   - Test:  (2104, 440)  -> 'test_data_reduced.csv'


In [17]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
import gc
model = joblib.load('final_melting_point_model.pkl')
features = joblib.load('final_features_list.pkl')

df = pd.read_csv('result/data/melting_point_features.csv')

needed_cols = list(features) + ['Tm']

existing_cols = [c for c in needed_cols if c in df.columns]

df_reduced = df[existing_cols].copy()

del df
gc.collect()

y = df_reduced['Tm']
X = df_reduced.drop(columns=['Tm'])

X = X.select_dtypes(include=[np.number])
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X = X.mask(X > 1e308, np.nan)

print("⚙️(Imputing)...")
imputer = SimpleImputer(strategy='median')
X_clean = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

_, X_test, _, y_test = train_test_split(X_clean, y, test_size=0.2, random_state=42)

y_pred = model.predict(X_test[features])

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("\n--- 🏁 RESULT ---")
print(f"RMSE: {rmse:.4f}")
print(f"R2: {r2:.4f}")

⚙️(Imputing)...

--- 🏁 RESULT ---
RMSE: 165.3967
R2: 0.8457


In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
import warnings

warnings.filterwarnings('ignore')

df = pd.read_csv('result/data/melting_point_features.csv')
y = df['Tm']
X = df.drop(columns=['Tm']).select_dtypes(include=[np.number])

X.replace([np.inf, -np.inf], np.nan, inplace=True)
X = X.mask(X > 1e308, np.nan)

imputer = SimpleImputer(strategy='median')
X_clean = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

X_train, X_test, y_train, y_test = train_test_split(X_clean, y, test_size=0.2, random_state=42)

def get_metrics(name, feature_list):
    valid_feats = [f for f in feature_list if f in X_train.columns]
    
    if not valid_feats: return {"Method": name, "Features": 0, "RMSE": 0, "R2": 0}

    model = LGBMRegressor(random_state=42, n_jobs=1, verbose=-1)
    model.fit(X_train[valid_feats], y_train)
    y_pred = model.predict(X_test[valid_feats])
    
    return {
        "Method": name,
        "Features": len(valid_feats),
        "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
        "R2": r2_score(y_test, y_pred)
    }

feats_all = list(X_train.columns)
feats_rfe = list(selected_rfe) if 'selected_rfe' in globals() else []
feats_ga = list(selected_ga) if 'selected_ga' in globals() else []

results = []
results.append(get_metrics("Original", feats_all))
results.append(get_metrics("RFE", feats_rfe))
results.append(get_metrics("GA", feats_ga))

df_res = pd.DataFrame(results)
base_rmse = df_res.loc[0, 'RMSE']
base_r2 = df_res.loc[0, 'R2']

df_res['Diff_RMSE'] = df_res['RMSE'] - base_rmse
df_res['Diff_R2'] = df_res['R2'] - base_r2

print(df_res.round(4))

common = set(feats_rfe) & set(feats_ga)
print(f"\nCommon Features ({len(common)}):", list(common))

     Method  Features      RMSE      R2  Diff_RMSE  Diff_R2
0  Original       937  264.6191  0.6050     0.0000   0.0000
1       RFE        30  282.3845  0.5502    17.7654  -0.0548
2        GA       439  269.2039  0.5912     4.5848  -0.0138

Common Features (19): ['Gasteiger_q_std', 'Flexibility_Score', 'MolMR', 'EState_VSA2', 'TPSA', 'SlogP_VSA2', 'SMR_VSA10', 'MolWt', 'FracDouble', 'NumValenceElectrons', 'BCUT2D_MRHI', 'qed', 'MinEStateIndex', 'SPS', 'FracSingle', 'Kappa3', 'HallKierAlpha', 'Chi0v', 'BalabanJ']
